# GoogLeNet
有没有一种方法既能保持网络结构的稀疏性，又能利用密集矩阵的高计算性能。大量的文献表明可以将稀疏矩阵聚类为较为密集的子矩阵来提高计算性能，就如人类的大脑是可以看做是神经元的重复堆积，因此，GoogLeNet团队提出了Inception网络结构，就是构造一种“基础神经元”结构，来搭建一个稀疏性、高计算性能的网络结构。

## 导入库

In [ ]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## 超参

In [ ]:
EPOCHS = 10
BATCH_SIZE = 64
VERBOSE = 1
IMAGE_SIZE = 224

## 导入数据集

In [ ]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.fashion_mnist.load_data()

In [ ]:
train_images = tf.reshape(train_images, (train_images.shape[0], train_images.shape[1], train_images.shape[2], -1))
test_images = tf.reshape(test_images, (test_images.shape[0], test_images.shape[1], test_images.shape[2], -1))

In [ ]:
train_images = tf.image.resize_with_pad(train_images, IMAGE_SIZE, IMAGE_SIZE)
test_images = tf.image.resize_with_pad(test_images, IMAGE_SIZE, IMAGE_SIZE)

In [ ]:
train_images = tf.cast(train_images, dtype=tf.float32) / 255.
test_images = tf.cast(test_images, dtype=tf.float32) / 255.

In [ ]:
train_labels = tf.convert_to_tensor(train_labels, dtype=tf.int32)
test_labels = tf.convert_to_tensor(test_labels, dtype=tf.int32)

## 定义模型

In [ ]:
class Inception(keras.layers.Layer):
    def __init__(self,c1, c2, c3, c4):
        super().__init__()
        self.p1_1 = keras.layers.Conv2D(c1, kernel_size=1, activation='relu', padding='same')
        self.p2_1 = keras.layers.Conv2D(c2[0], kernel_size=1, padding='same', activation='relu')
        self.p2_2 = keras.layers.Conv2D(c2[1], kernel_size=3, padding='same', activation='relu')
        self.p3_1 = keras.layers.Conv2D(c3[0], kernel_size=1, padding='same', activation='relu')
        self.p3_2 = keras.layers.Conv2D(c3[1], kernel_size=5, padding='same', activation='relu')
        self.p4_1 = keras.layers.MaxPool2D(pool_size=3, padding='same', strides=1)
        self.p4_2 = keras.layers.Conv2D(c4, kernel_size=1, padding='same', activation='relu')

    def call(self, x):
        p1 = self.p1_1(x)
        p2 = self.p2_2(self.p2_1(x))
        p3 = self.p3_2(self.p3_1(x))
        p4 = self.p4_2(self.p4_1(x))
        return tf.concat([p1, p2, p3, p4], axis=-1)

In [ ]:
b1 = keras.models.Sequential()

b1.add(keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same', activation='relu'))
b1.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same'))

b2 = keras.models.Sequential()
b2.add(keras.layers.Conv2D(64, kernel_size=1, padding='same', activation='relu'))
b2.add(keras.layers.Conv2D(192, kernel_size=3, padding='same', activation='relu'))
b2.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same'))

b3 = keras.models.Sequential()
b3.add(Inception(64, (96, 128), (16, 32), 32))
b3.add(Inception(128, (128, 192), (32, 96), 64))
b3.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same'))

b4 = keras.models.Sequential()
b4.add(Inception(192, (96, 208), (16, 48), 64))
b4.add(Inception(160, (112, 224), (24, 64), 64))
b4.add(Inception(128, (128, 256), (24, 64), 64))
b4.add(Inception(112, (144, 288), (32, 64), 64))
b4.add(Inception(256, (160, 320), (32, 128), 128))
b4.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same'))

b5 = keras.models.Sequential()
b5.add(Inception(256, (160, 320), (32, 128), 128))
b5.add(Inception(384, (192, 384), (48, 128), 128))
b5.add(keras.layers.GlobalAvgPool2D())

model = keras.models.Sequential([b1, b2, b3, b4, b5, tf.keras.layers.Dense(10)])

## 设置优化器和损失函数

In [ ]:
optimizer = keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## 开始训练

In [ ]:
history = model.fit(train_images, train_labels, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=VERBOSE)

## 绘制梯度下降图

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid(True)
    plt.gca().set_ylim(0, 0.1)
    plt.show()
    
plot_learning_curves(history)

## 测试训练好的模型

In [ ]:
model.evaluate(test_images, test_labels)

## 保存模型 

In [ ]:
model.save('./save/save_model/model.h5')

In [ ]:
model.save_weights('./save/save_weights/model_weights.h5')